In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 3.6 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=220e7d9e4e951cff071222347812fcf7846a970664d9a9f3d48fc0eea6ffcb59
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

ln: failed to create symbolic link '/mydrive/MyDrive': File exists
'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [5]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [6]:
!ls

 a0529-a0611_US         dataframe_csv_UK
 a0529.csv	        dataframe_result_csv
 a0530.csv	        dataframe_result_csv_UK
 a0531.csv	        Github
 a0601.csv	        heatmap529_UK.ipynb
 a0602.csv	        heatmap530.ipynb
 a0603.csv	        Heat_Map_Online_language.ipynb
 a0604.csv	        __MACOSX
 a0605.csv	        non_violant1.csv
 csv		        non_violant2.csv
 csv.zip	        UK
 dataframe0603_UK.csv   violant.csv
 dataframe0604_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv


In [7]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
ct_path_531 = 'UK/06_05.csv'

In [9]:
ct = pd.read_csv(ct_path_531)

In [10]:
ct.shape[1]

47

In [11]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [12]:
#len(columns1)
len(columns2)

47

In [13]:
ct.columns=columns2

In [14]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814
1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344
2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737
3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804
4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35409,460670040801275_14339257

In [15]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [16]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [17]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [20]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [21]:
ct['NER-msg'] = ct['message'].apply(ner)

In [22]:
ct['NER-desc'] = ct['description'].apply(ner)

In [23]:
ct['NER-title'] = ct['title'].apply(ner)

In [24]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Ada': 1},{}
1,{},{}
2,{},{'Canada': 2}
3,{'Minneapolis': 2},{'Canada': 2}
4,{},{'Canada': 2}


In [25]:
ct.to_csv('a0605.csv')

In [29]:
ct=pd.read_csv('a0605.csv',lineterminator="\n")

In [30]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [31]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Ada': 1},{},{}
2,{},{'Canada': 2},{}
3,{'Minneapolis': 2},{'Canada': 2},{}
4,{},{'Canada': 2},{}
5,{},{'covid': 1},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [34]:
ct_ner['NER-msg']

0                     [Ada]
2                        []
3             [Minneapolis]
4                        []
5                        []
                ...        
35403                    []
35408                    []
35411                    []
35412    [Lebanon, America]
35413                    []
Name: NER-msg, Length: 24068, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ct_ner['NER']

0                                                    [Ada]
2                                                 [Canada]
3                                    [Minneapolis, Canada]
4                                                 [Canada]
5                                                  [covid]
                               ...                        
35403    [Ohio, Minneapolis, Arizona, Iowa, California,...
35408    [Ohio, Minneapolis, Arizona, Iowa, California,...
35411              [Iowa City, Iowa, IOWA CITY, Iowa City]
35412                 [Lebanon, America, Lebanon, America]
35413                                   [Lebanon, America]
Name: NER, Length: 24068, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
ner_lst = ct_ner['NER'].values

In [40]:
ner_lst

array([list(['ada']), list(['canada']), list(['minneapolis', 'canada']),
       ..., list(['iowa city', 'iowa', 'iowa city']),
       list(['lebanon', 'america']), list(['lebanon', 'america'])],
      dtype=object)

In [41]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [42]:
list

['ada',
 'canada',
 'minneapolis',
 'canada',
 'canada',
 'covid',
 'u.s.',
 'denver',
 'denver',
 'minneapolis',
 'denver',
 'denver',
 'wisconsin',
 'denver',
 'santa barbara',
 'denver',
 'denver',
 'denver',
 'america',
 'denver',
 'byron schlomach',
 'denver',
 'annapolis',
 'minneapolis',
 'minnesota',
 'dallas',
 'minnesota',
 'arlington',
 'va',
 'va',
 'the united states',
 'minneapolis',
 'arlington',
 'fentonyl',
 'arlington',
 'va',
 'va',
 'u.s.',
 'wales',
 'arlington',
 'louisville',
 'england',
 'kentucky',
 'arlington',
 'va',
 'oak lawn',
 'oak lawn',
 'oak lawn',
 'oak lawn',
 'oak lawn',
 'wy',
 'oak lawn',
 'san francisco’s',
 'america',
 'iran',
 'china',
 'us',
 'usa',
 'america',
 'sf',
 'the united states’',
 'montpelier',
 'new york',
 'mudville',
 'minneapolis',
 'the united states',
 'montpelier',
 'montpelier',
 'arlington',
 'va',
 'montpelier',
 'arlington',
 'va',
 'ky.',
 'va',
 'the united states',
 'minneapolis',
 'indianapolis',
 'arlington',
 'montp

In [43]:
c = collections.Counter()

In [44]:
dict_a=collections.Counter(list).most_common()

In [45]:
dict_a

[('minneapolis', 7327),
 ('america', 3037),
 ('us', 2497),
 ('the united states', 1633),
 ('minnesota', 1442),
 ('u.s.', 1425),
 ('washington', 1008),
 ('texas', 854),
 ('new york', 752),
 ('chicago', 734),
 ('houston', 680),
 ('d.c.', 605),
 ('new york city', 548),
 ('buffalo', 478),
 ('usa', 471),
 ('los angeles', 468),
 ('california', 377),
 ('maryland', 358),
 ('uk', 355),
 ('ghana', 342),
 ('canada', 329),
 ('nyc', 288),
 ('atlanta', 284),
 ('london', 254),
 ('dc', 245),
 ('florida', 243),
 ('dallas', 216),
 ('georgia', 193),
 ('north carolina', 188),
 ('china', 187),
 ('michigan', 187),
 ('australia', 185),
 ('detroit', 182),
 ('brooklyn', 180),
 ('hollywood', 177),
 ('denver', 169),
 ('alabama', 166),
 ('portland', 166),
 ('louisville', 164),
 ('seattle', 164),
 ('new jersey', 159),
 ('the united states of america', 154),
 ('virginia', 146),
 ('united states', 143),
 ('philadelphia', 142),
 ('new zealand', 135),
 ('st. louis', 128),
 ('oakland', 126),
 ('germany', 125),
 ('israe

In [46]:
dataframe531=pd.DataFrame(dict_a)

In [47]:
len(dict_a)

4651

In [48]:
dataframe531.to_csv('dataframe0605_UK.csv')